In [1]:
%load_ext autoreload   

In [2]:
k = 1000
model = 'gpt-4o-mini'

In [5]:
%load_ext autoreload
%autoreload 2

import asyncio
import pandas as pd
from vector_search import VectorSearch, build_context_string
from typing import List

from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens

from vexa import VexaAPI
from core import system_msg, user_msg,generic_call_stream
from datetime import datetime

from prompts import Prompts
prompts = Prompts()


from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens, assistant_msg
from pydantic_models import QueryPlan, Query
from thread_manager import ThreadManager
from pydantic_models import ThreadName

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
vexa = VexaAPI()
analyzer = VectorSearch()
thread_manager = ThreadManager()
print(analyzer.count_documents(user_id=vexa.user_id))


User information retrieved successfully.
1145


In [21]:
thread_id = thread_manager.upsert_thread(user_id=vexa.user_id, thread_name="test11", messages=[user_msg("test")])
thread_id

Thread upserted with id 924941ff-22b9-4f22-8929-a8ac5fd0a8f2


'924941ff-22b9-4f22-8929-a8ac5fd0a8f2'

In [22]:
thread_manager = ThreadManager()
threads = thread_manager.get_user_threads(vexa.user_id)


In [23]:
len(threads)


10

In [7]:
async def generate_output(query, user_id=None, user_name=None, thread_id=None):
    # Initialize ThreadManager
    thread_manager = ThreadManager()
    
    # Get or create thread and messages
    if thread_id:
        thread = thread_manager.get_thread(thread_id)
        if not thread:
            raise ValueError(f"Thread with id {thread_id} not found")
        messages = thread.messages
        thread_name = thread.thread_name
    else:
        messages = []
        thread_name = None

    # Generate search queries
    query_ = ' '.join([m.content for m in messages]) + ' ' + query
    queries = await analyzer.generate_search_queries(query_, user_id=user_id, user_name=user_name)
    print(queries)
    
    if not user_id and not user_name:
        raise ValueError("Either user_id or user_name must be provided")
    
    summaries = analyzer.get_summaries(user_id=user_id, user_name=user_name)
    full_context, meeting_ids = await analyzer.build_context(queries, summaries, include_all_summaries=False, user_id=user_id, user_name=user_name)
    
    pref = "Based on the following context, answer the question:" if len(messages) == 0 else "Follow-up request:"
    user_info = f"The User is {user_name}"
    messages_context = [
        system_msg(prompts.perplexity + f'. {user_info}'), 
        user_msg(f"Context:\n{full_context}")
    ] + messages + [user_msg(f"{pref} {query}")]
    
    output = await generic_call_stream(messages_context)
    messages.append(user_msg(query))
    messages.append(assistant_msg(output))

    # Create or update thread
    if not thread_id:  # New conversation
        messages_str = ';'.join([m.content for m in messages if m.role == 'user'])
        thread_name = await ThreadName.call([user_msg(messages_str)])
        thread_id = thread_manager.create_thread(user_id=user_id, thread_name=thread_name[0].thread_name, messages=messages)
    else:  # Existing conversation
        thread_manager.update_thread(thread_id, messages)

    return output, messages, meeting_ids, full_context, thread_id, thread_name

In [12]:
thread_id = threads[1]['thread_id']

In [13]:
thread_manager.get_messages_by_thread_id(thread_id)

[Msg(role='user', content='who do i work with?', stage=None),
 Msg(role='assistant', content='You work with various individuals, including Umar Lateef, Tatiana Sukhova, Rick Tousseyn, Olga Nemirovskaya, Angelina Geru, and Igor Bessonov, among others. Your discussions cover a range of topics such as social media strategies, product development, marketing strategies, and technical troubleshooting.', stage=None)]

In [14]:
query = 'update'
output, messages,meeting_ids,full_context,thread_id,thread_name = await generate_output(query,user_id=vexa.user_id,user_name=vexa.user_name,thread_id=thread_id)

[{'start': None, 'end': None, 'vector_search_query': 'collaboration with Umar Lateef on VEXA product strategy and user feedback, discussing market strategies and pricing plans for user acquisition.'}, {'start': None, 'end': None, 'vector_search_query': 'Tatiana Sukhova and Dmitriy Grankin discussing branding and investment strategies, focusing on design challenges and innovative investment models.'}, {'start': None, 'end': None, 'vector_search_query': 'Rick Tousseyn and Dmitriy Grankin discussing SEO tool development, focusing on content creation tools and SEO optimization strategies.'}, {'start': None, 'end': None, 'vector_search_query': 'Olga Nemirovskaya and Dmitriy Grankin discussing product launch strategies on Product Hunt, emphasizing promotional tactics and engagement metrics.'}, {'start': None, 'end': None, 'vector_search_query': 'Angelina Geru sharing insights on content creation on LinkedIn, focusing on challenges and expectations for new content creators.'}, {'start': None,

In [11]:
messages

[Msg(role='user', content='who are they?', stage=None),
 Msg(role='assistant', content='The context primarily features Dmitriy Grankin, who is involved in various discussions related to social media strategies, product development, marketing strategies, and technical troubleshooting. He collaborates with several individuals, including Umar Lateef, Tatiana Sukhova, Rick Tousseyn, Olga Nemirovskaya, and Igor Bessonov, among others. These individuals appear to be colleagues or partners in different projects, focusing on areas such as marketing, product development, SEO strategies, and AI-driven solutions.', stage=None),
 Msg(role='user', content='list', stage=None),
 Msg(role='assistant', content='- **Dmitriy Grankin**: Central figure in discussions about social media strategies, product development, and marketing.\n\n- **Umar Lateef**: Collaborates with Dmitriy on market strategy and user feedback.\n\n- **Tatiana Sukhova**: Engages in discussions about branding and investment strategies.

In [ ]:
thread_id

'bb847436-3d25-4d56-a6d3-ed4bc7278dc2'

In [ ]:
thread_name

'What is going on?'

In [ ]:
messages

[Msg(role='user', content='what is going on?', stage=None),
 Msg(role='assistant', content='The technical team is engaged in ongoing discussions and updates regarding various projects, focusing on bug fixes, smart contract modifications, and integration tasks. Key topics include the implementation of a cache system, collaboration with Sign Protocol, and addressing issues related to WebSocket and frontend/backend development. Team members are actively working on resolving technical challenges, such as connection issues with Cloudflare workers and problems with smart contracts. There is an emphasis on collaboration, code reviews, and the importance of meeting deadlines for project completion. Additionally, the team is preparing documentation and proposals for upcoming tasks and features, indicating a proactive approach to project management and development.', stage=None),
 Msg(role='user', content='what is the company profile?', stage=None),
 Msg(role='assistant', content='The company ap

In [ ]:
from assistant_thread import SearchAssistantThreadManager

In [ ]:

thread_manager = SearchAssistantThreadManager()

# Then you can use it as before:
thread_id = thread_manager.create_thread(user_id="user123", thread_name="My First Thread1", messages=messages,thread_id=thread_id)

In [ ]:
thread_id

'73ace234-2c88-4d4b-bc2a-24bdbe1691fc'

In [ ]:
thread_manager.get_thread(thread_id)

SearchAssistantThread(thread_id='73ace234-2c88-4d4b-bc2a-24bdbe1691fc', user_id='user123', thread_name='My First Thread1', messages=[Msg(role='user', content='who do i work with?', stage=None), Msg(role='assistant', content='You work with several individuals, including Umar Lateef, Tatiana Sukhova, Rick Tousseyn, Olga Nemirovskaya, Angelina Geru, and Igor Bessonov. Your discussions cover various topics such as social media strategies, product development, marketing strategies, and technical troubleshooting.', stage=None)], timestamp=datetime.datetime(2024, 10, 17, 15, 20, 46, 572578))

In [ ]:
messages

[Msg(role='user', content='who is the user?', stage=None),
 Msg(role='assistant', content='The user is Dmitriy Grankin.', stage=None),
 Msg(role='user', content='who do i work with?', stage=None),
 Msg(role='assistant', content='You work with Sergey Ryabenko, as indicated in the discussions regarding the Figma plugin for HTML generation and other technical meetings.', stage=None),
 Msg(role='user', content='who do i work with?', stage=None),
 Msg(role='assistant', content='You work with Sergey Ryabenko, as well as other team members involved in various discussions and testing sessions.', stage=None)]

In [ ]:
print(full_context)

# [1] Figma Plugin Discussion (2024-10-14 13:21):
In a technical meeting on October 14, 2024, Sergey Ryabenko and Dmitriy Grankin discussed the use of a Figma plugin for HTML generation. They addressed issues with the plugin's performance, the need for responsive design, and the potential use of Tailwind CSS for styling. The conversation included troubleshooting steps and the importance of exporting code effectively.
## Related Quotes:
:  Все, Дим, давай. :  Нет, попробуем, попробуем сперва, попробуем. :  Давай, пока.
Dmitriy Grankin:  export code get pro давай export Sergey Ryabenko:  download zip Dmitriy Grankin:  Не, окей, а посмотреть-то можно, что получилось? Dmitriy Grankin:  Ну вот ты смотришь, видимо. Dmitriy Grankin:  Так я же хочу посмотреть на код. Sergey Ryabenko:  Ну, сделай инспект. Sergey Ryabenko:  А-а-а. Dmitriy Grankin:  Логично. Sergey Ryabenko:  Ты можешь, знаешь, что попробовать? Sergey Ryabenko:  Подожди, кликни правой кнопкой, там должно быть... Sergey Ryabenko: 

In [ ]:
count_tokens(full_context)

8835

In [ ]:
30000*100/1000000*0.15

0.44999999999999996

In [ ]:
query = 'list all condidate names considered and your assessment'
output, messages,meeting_ids,full_context = await generate_output(query, messages)

[{'start': None, 'end': None, 'vector_search_query': 'candidate evaluation for backend positions, concerns about qualifications and fit, ongoing evaluation process, overqualified candidates, coding abilities assessment'}, {'start': None, 'end': None, 'vector_search_query': 'status of backend position, open positions, candidate qualifications, technical role evaluations, long-term fit for team'}, {'start': None, 'end': None, 'vector_search_query': 'names of candidates considered for backend roles, assessment of coding skills, theoretical knowledge vs practical abilities, ongoing evaluations'}, {'start': None, 'end': None, 'vector_search_query': 'discussions on backend candidate evaluations, concerns about overqualified candidates, importance of practical coding skills, team fit'}, {'start': None, 'end': None, 'vector_search_query': 'current status of backend position, candidate evaluation process, qualifications of candidates, ongoing search for suitable candidates'}]
The discussions in

In [ ]:
query = 'specific names please'
output, messages,meeting_ids = await generate_output(query, messages)

[{'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'candidates for backend development'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'backend developer position status'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'discussion on backend candidates'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'backend developer hiring process'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'October 14 meeting on backend candidates'}]
The context does not provide any specific names of candidates considered for the backend position. The discussions mainly revolved around technical challenges and the need for developers w

In [ ]:

meeting_id_ref = 4
dashboard_url = f"https://dashboard.vexa.ai/#{meeting_ids[meeting_id_ref-1]}"
dashboard_url


'https://dashboard.vexa.ai/#8ad52667-1a1d-4aa6-ba9a-a2021fe650bc'